## libraries needed

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import spacy
import nltk
import re
from text_preprocessing import preprocess_text
from text_preprocessing import to_lower, remove_email, remove_url, lemmatize_word,expand_contraction
from text_preprocessing import remove_punctuation as punct
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
from scispacy.abbreviation import AbbreviationDetector
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from collections import Counter
#!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')
## importing libraries
from nltk.tokenize import TreebankWordTokenizer 
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
#post-Processing
import gensim
import matplotlib.pyplot as plt
#from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import json
import os
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import nltk
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cluster
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy.cluster import hierarchy
#from sklearn.cluster import AgglomerativeClustring
from rouge_score import rouge_scorer
!pip install abbreviations
from abbreviations import schwartz_hearst
import statistics as st

## path of the dataset DUC2002

In [3]:
import re
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import os
doc_path = "DucDataset/DUC2002_Summarization_Documents/docs"
doc_sent = "DucDataset/DUC2002_Summarization_Documents/docs.with.sentence.breaks"
summ1_path = "DucDataset/DUC2002_Summarization_Documents/summaries"
files = os.listdir(doc_path)


## reading in the different types of file each has it's own way

In [5]:
def parse_perdocs_xml(path):

    with open(path, "r") as fp:
        xml = fp.read()

    header_patt = r'<SUM.*?TYPE="PERDOC"\s+SIZE="(.*?)"\s+DOCREF="(.*?)"' \
        '\s+SELECTOR="(.*?)"\s+SUMMARIZER="(.*?)"\s*>'
    summary_patt = header_patt + r'(.*?)</SUM>'

    headers = [m for m in re.findall(header_patt, xml, flags=re.DOTALL)]
    headers_found = len(headers)

    data = {}
    for match in re.findall(summary_patt, xml, flags=re.DOTALL):
        size, doc_id, selector, summarizer, summary_text = match
        doc_id = doc_id.strip()
        sentences = summary_text
        sentences = sentences.strip().replace('\n', ' ')
        
        sentences = sent_tokenize(sentences)
        data[doc_id] = sentences

    return data

def parse(path):
    with open(path, "r") as fp:
        file = fp.read()
    soup = BeautifulSoup(file)
    text = []
    for a in soup.find_all('text'):
        for s in a.find_all('s'):
            text.append(s.string)
    return text
    
def parse_input_docs(paths,fn,text_path):

    if fn.startswith("WSJ"):
        doc_id, Headline, text , t  = parse_wsj(paths)
    elif fn.startswith("SJMN"):
        doc_id, Headline, text,t  = parse_sjmn(paths)
    elif fn.startswith("FT"):
        doc_id, Headline, text = parse_ft(paths)
    elif fn.startswith("AP"):
        doc_id, Headline, text  = parse_ap(paths)
    elif fn.startswith("LA"):
        doc_id, Headline, text  = parse_la(paths)
    elif fn.startswith("FBIS"):
        doc_id, Headline, text  = parse_fbis(paths)
    else:
        doc_id=" "
        Headline = " "
        text = " " 
    if text != " ":
        text = parse(text_path)
    if fn.startswith("WSJ") or fn.startswith("SJMN"):
        if text != " ":
            text = t + text 
        else:
            text = t


    data = {"doc_id": doc_id,"title": Headline ,"input_data": text}

    return data

def parse_fbis(file_path):

    sentences = []

    with open(file_path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HEADER>(.*?)</HEADER>+', file)
    if len(Headline[0].split('Document Title:')) > 1:
        Headline = Headline[0].split('Document Title:')[-1]
    else:
        Headline = []
    
    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = re.sub(r"</?F.*?>", r" ", text) 
    text = sent_tokenize(text)       
    return doc_id, Headline, text 

def parse_sjmn(path):

    sentences = []
    with open(path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HEADLINE>(.*?)</HEADLINE>+', file)
    headline = Headline
    leadpara = re.findall(r'<LEADPARA>(.*?)</LEADPARA>+', file)[0]
    leadpara = sent_tokenize(leadpara)
    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = sent_tokenize(text)
    out = text
    return doc_id, headline, out, leadpara

def parse_wsj(path):

    sentences = []

    with open(path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HL>(.*?)</HL>+', file)
    headline = Headline
    t = re.findall(r'<LP>(.*?)</LP>+', file)

    if t :
        t = sent_tokenize(t[0])
    else :
        t = []
    
    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = sent_tokenize(text)
            
    return doc_id, headline, text ,t

def parse_ft(path):

    sentences = []

    with open(path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HEADLINE>(.*?)</HEADLINE>+', file)[0]
    headline = Headline.split("/")[1:]
    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = sent_tokenize(text)
            
    return doc_id, headline, text 

def parse_ap(path):

    sentences = []

    with open(path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HEAD>(.*?)</HEAD>+', file)
    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = sent_tokenize(text)
            
    return doc_id, Headline, text 
def parse_la(file_path):

    sentences = []

    with open(file_path, "r") as fp:
        file = fp.read()
        fp.closed

    file = file.strip().replace('\n', ' ')
    doc_id = re.findall(r'<DOCNO>(.*?)</DOCNO>+', file)[0]
    Headline = re.findall(r'<HEADLINE>(.*?)</HEADLINE>+', file)
    Headline = re.findall(r'<P>(.*?)</P>+', Headline[0])

    text = re.findall(r'<TEXT>(.*?)</TEXT>+', file)[0]
    text = re.findall(r'<P>(.*?)</P>+', text)
    #text = tokenizer.tokenize(text)
            
    return doc_id, Headline, text 



In [6]:
ref1_files = os.listdir(summ1_path)
Summaries = {}
ref_data1 = {}
for file in ref1_files:
    for files in os.listdir( summ1_path + "/" + file):
        if files == "perdocs":
            check = summ1_path + "/" + file + "/perdocs"
            data =  parse_perdocs_xml(check)
            input_path = doc_path + "/" +file[:-1]
            in_ids = os.listdir(input_path)
            ref_data1[file] = data
            for key,value in data.items():
                doc_id = key
                if doc_id in Summaries.keys():
                    Summaries[doc_id].append(value)
                else:
                    Summaries[doc_id] = [value]


In [7]:
files = os.listdir(doc_path)
file_data = []
check_dups =[]
Text_in = {}
for file in files:
    file_path = doc_path + "/" + file
    in_files = os.listdir(file_path)
    for In_in_file in in_files:
        file_path = doc_path + "/" + file + "/" + In_in_file
        text_path = doc_sent + "/" + file + "/" + In_in_file + ".S"
        In_data = parse_input_docs(file_path, In_in_file,text_path)
        doc_id = In_data['doc_id'].strip()
        title = In_data['title']
        input_data = In_data['input_data']
        if doc_id not in check_dups:
            Text_in[doc_id] = {'doc_id': doc_id, 'title': title, 'data_input': input_data}
            check_dups.append(doc_id)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
nlp = spacy.load("en_core_web_lg")
#from spacy_langdetect import LanguageDetector
from spacy.language import Language
abbreviation_pipe = AbbreviationDetector(nlp)
def create_lang_detector(nlp, name):
    return abbreviation_pipe
Language.factory("abbreviation_pipe", func=create_lang_detector)
nlp.add_pipe("abbreviation_pipe")

## for extracting variables needed from the input

In [10]:
def read_input(i,doc_id):

    title_line = i['title']
    text = i['data_input']
    
    Summary = Summaries[doc_id]
    
    title = []
    keywords =[]
    if(len(title_line) != 0):
        for i in title_line:
            title = title + word_tokenize(i)
            
    title = [item.lower() for item in title]
        
    lang_Dict = []
    for line in text:
        for word in line.split(" "):
            if word not in lang_Dict:
                lang_Dict.append(word)
    iter_dict = lang_Dict
    lang_Dict = []
    for i in iter_dict:
        if '\n' in i:
            lang_Dict = lang_Dict + i.split("\n")
        else:
            lang_Dict.append(i)
     
    remove = string.punctuation
    remove = remove.replace("-", "") # don't remove hyphens
    pattern = r"[{}]".format(remove)
    
    lang_dict = []
    for word in lang_Dict:
        if word.strip():
            word = word.lower()
            word = expand_contraction(word)
            word = re.sub(pattern, '', word)
            lang_dict.append(word)
            
    lang_Dict = []
    for word in lang_dict:
        if " " in word:
            for w in word.split(" "):
                if w.strip():
                    lang_Dict.append(w)
        else:
            lang_Dict.append(word)
    keyvalue = dict() 

    index= 0
    for line in text:
        if(line and (line.strip()) ):
            i = index
            keyvalue[i] = line
            index = index + 1

    key_value_list =[]
    for item in keyvalue.items():
        key_value_list.append(item)
    
    return title,keywords,keyvalue, key_value_list,Summary, lang_Dict
title,keywords,keyvalue, key_value_list,Summary, lang_Dict = read_input(Text_in['AP880911-0016'],'AP880911-0016')
#print(keywords)

## functions needed for preprocessing

In [12]:
def replace_acronymss(keyvalue):
        sent_dict = dict() 
        whole_text = ""
        for key, text in keyvalue.items():
            whole_text = whole_text + "#*# " + text
        whole_text = whole_text.lower()
        pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text=whole_text, most_common_definition=True)
        for key,value in pairs.items():
            whole_text = whole_text.replace(value, key)
        output_text = whole_text.split("#*# ")
        i = 0;
        for l in output_text:
            if l != "":
                sentence = Brackets(l)
                sent_dict[i] = sentence
                i = i+1
        return sent_dict

#remove hyphens and tokenizing sentence
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        word = expand_contraction(word)
        if word.lower() == "n't":
            word = "not"
        if word != '':
            word = word.lower()
            #word = expand_contraction(word)
            new_words.append(word)
    #print(words)
    #print("after remove punck")
    return new_words

def tokenize_sent(sent_dict):
    keyvalueToken = dict() 
    TorF = True
    for key, value in sent_dict.items():
        
        words = nltk.word_tokenize(expand_contraction(value))
        words = [re.sub(r'(\n-)','',word) for word in words]
        words = [re.sub(r'(\n)',' ',word) for word in words]
        new_words = []
        for word in words:
            if " " in word:
                 new_words = new_words + word.split(" ")
            else:
                new_words.append(word)
        
        keyvalueToken[key] = new_words
        
    return keyvalueToken
#continue sent reduction
def conjunctions(tokenize_sent_dict):
    new_sent_dict = dict()
    conjunctive_adverbs= ["accordingly","comparatively","contrarily", 
                      "also"," further", "nevertheless", "indeed"," otherwise",
                      "result","as","equally" ,"conversely" ,"besides",
                      "furthermore" ,"nonetheless" ,"fact" ,"consequently",
                     " likewise" ,"however" ,"addition" ",moreover" ,"surprisingly",
                     " hence" ,"similarly ","comparison" ,"still ","therefore ",
                      "contrast" ,"thus" ,"instead"  ,"rather"
                      , "other", "additionally", "addition",'although','already',"including",'according',
                    'totally','likely','unlikely','perhaps',"s",'naturally',
                      "finally", "based","on","that" , "besides","namely","anyway",
                     "then","next","thereafter","certainly","now","finally","meanwhile",
                     "subsequently","yet","elsewhere","thereafter","undoubtedly",
                     "incidentally","otherwise","regardless","begin","in"]
    
    for key, text in tokenize_sent_dict.items():
        new_sent = [word for word in text if word.lower() not in conjunctive_adverbs]
        new_sent_dict[key] = new_sent

    return new_sent_dict

#d Sentence reduction 
def Brackets(text):
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    return text

def get_key(val,my_dict): 
    for key, values in my_dict.items(): 
        for value in values:
            if val == value: 
                return key 

    return "key doesn't exist"

def replace_word_synonym(new_sent_dict):
    
    text = []
    sent_dict = dict()
    dict_synonym = Synonym_replace(new_sent_dict)
    for key, value in new_sent_dict.items():
        
        text = []
        for word in value:
            if get_key(word,dict_synonym) != "key doesn't exist":
                text.append(get_key(word,dict_synonym))
            else:
                text.append(word)
        sent_dict[key] = text
    
    return sent_dict

def Synonym_replace(new_sent_dict):
    synonymList = []
    text = {}
    for key, sentence in new_sent_dict.items():
        for word in sentence:
            if word not in synonymList:
                wordNetSynset =  wn.synsets(word)
                values = []
                for synSet in wordNetSynset:
                    for synWords in synSet.lemmas():
                        if synWords.name() in synonymList:
                            continue
                        else:
                            synonymList.append(synWords.name())
                            values.append(synWords.name())
                text[word] = values
    return text
#f Use words N-grams
def ngramise(sequence):
    bigrams = []
    for bigram in nltk.ngrams(sequence, 2):
        bigrams.append(bigram)
    return bigrams


def remove_hyfen(wordslist):
    words_new = []
    for word in wordslist:
        digit = False
        lang = False
        if "-" in word:
            words = word.split("-")
            for w in words:
                if w.isdigit():
                    digit = True
            if digit:
                newwords = [w for w in words]
                words_new = words_new + newwords
            else:
                for w in words:
                    if w in lang_Dict:
                        lang = True
                if lang:
                    newwords = [w for w in words]
                    words_new = words_new + newwords
                else:
                    newwords = [word]
                    words_new = words_new + newwords
        else:
            newwords = [word]
            words_new = words_new + newwords
    return words_new
#4.3. Word frequency computation
def EF(sent_dict):
    final_sent = dict()
    all_words = []
    stopWords = set(stopwords.words('english'))
    #word_tokens = word_tokenize(text) 
    remove = string.punctuation
    remove = remove.replace("-", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    
    for key, value in sent_dict.items():
        value = remove_hyfen(value)
        value = remove_punctuation(value)
        filtered_sentence = [w for w in value if w not in stopWords] 
        #filtered_sentence = [re.sub(r'[" "]+', '', word) for word in filtered_sentence]
        #replace everything not in the list with space
        filtered_sentence = [re.sub("'", '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub('"', '', word.lower()) for word in filtered_sentence]
        filtered_sentence = [re.sub(pattern, '', word) for word in filtered_sentence]
        filtered_sentence = [re.sub(r'[0-9]+', '', word) for word in filtered_sentence]
        filtered_sentence = [word for word in filtered_sentence if len(word.strip()) >1]
        filtered_sentence = [word for word in filtered_sentence if (re.sub(r"[^a-zA-Z-]+", '', word).strip())]
        #filtered_sentence = [re.sub(r"[^a-zA-Z-]+", '', word) for word in filtered_sentence]
        filtered_sentence = [word.strip() for word in filtered_sentence]

        wordnet_lemmatizer = WordNetLemmatizer() 
            
        filtered_sentence = [wordnet_lemmatizer.lemmatize(word,get_wordnet_pos(word))for word in filtered_sentence if  word != " " and word != '']
        
        final_sent[key] = filtered_sentence
        all_words = all_words + filtered_sentence
    word_frequency = list(Counter(all_words).items())
    return word_frequency, final_sent , all_words
# Lemmatize with POS Tag
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


## preprocessing

In [13]:
def preprocess():
    
    #detection of appreviation
    sent_dict = replace_acronymss(keyvalue)

    tokenize_sent_dict = tokenize_sent(sent_dict)
    
    new_sent_dict = conjunctions(tokenize_sent_dict)
    
    #e Replacement of word synonyms
    sent_dict = replace_word_synonym(new_sent_dict)
    sent_list =[]
    for item in sent_dict.items():
        sent_list.append(item)

    all_bigrams = []
    for key, value in sent_dict.items():
        bigrams = ngramise(value)
        all_bigrams = all_bigrams+bigrams

    bi_count = dict(Counter(all_bigrams))
    delete = [key for key,value in bi_count.items() if value < 2]
    # delete the key 
    for key in delete: del bi_count[key]
    bi_count_list =[]
    for key , value in bi_count.items():
        bi_count_list.append([(key),value])
        
    
    word_freq ,final_sent, all_words = EF(sent_dict)
    Sentences_list = []
    for item in final_sent.items():
        Sentences_list.append(item)     
    
    #word frequencies
    #freq of words in title
    title_freq = Counter(title)

    Keyword_freq = Counter(keywords)
    #Keyword_freq = keywords
    #print(Keyword_freq)
    w_freq = Counter(all_words)
    final_Word_freq = dict()
    for word in all_words:
        if word not in title_freq.keys():
            title_freq[word] = 0
        if word not in Keyword_freq.keys():
            Keyword_freq[word] = 0
        final_Word_freq[word] = title_freq[word] + Keyword_freq[word] + w_freq[word] 

    ##final list of sentences after preprocessing
    final_input = []
    for (index, sentence) in Sentences_list:
        final_input.append(" ".join(sentence))
        
    keyword_word_freq = []
    for word in all_words:
        keyword_word_freq.append([word,Keyword_freq[word]])  
    #print(keyword_word_freq)
    title_word_freq = []
    for word in all_words:
        title_word_freq.append([word, title_freq[word]])

    Sentences_list
    List_of_sent = dict()
    for (key, sent) in Sentences_list:
        List_of_sent[key] = " ".join(sent)

    final_Word_freq
    finalwordfreq = []
    for key , value in final_Word_freq.items():
        finalwordfreq.append([key,value])
    #finalwordfreq
    return (title,keywords,Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
            ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq)
    
(title,keywords,Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
 ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq) = preprocess() 

def bigrams_freq(bi_count_list):
    words_freq_bigram = dict()
    bi_count = []
    for list_bi in bi_count_list:
        (w1,w2), i = list_bi
        if w1 in words_freq_bigram.keys():
            words_freq_bigram[w1] = words_freq_bigram[w1] + i
        else:
            words_freq_bigram[w1] = i
        if w2 in words_freq_bigram.keys():
            words_freq_bigram[w2] = words_freq_bigram[w2] + i
        else:
            words_freq_bigram[w2] = i   
    for key,value in words_freq_bigram.items():
        bi_count.append([key,value])
    return bi_count
bi_count = bigrams_freq(bi_count_list)

In [15]:
words_freq_bigram = dict()
for list_bi in bi_count_list:
    (w1,w2), i = list_bi
    if w1 in words_freq_bigram.keys():
        words_freq_bigram[w1] = words_freq_bigram[w1] + i
    else:
        words_freq_bigram[w1] = i
    if w2 in words_freq_bigram.keys():
        words_freq_bigram[w2] = words_freq_bigram[w2] + i
    else:
        words_freq_bigram[w2] = i 

## weight computation

In [16]:
#weights
def get_weights(list_of_nodes):
    import pandas as ps
    import statistics as st
    node_weight_dict = dict()
    node_list= list_of_nodes
    FWL= finalwordfreq
    TWL= title_word_freq
    KWL= keyword_word_freq
    BWL= bi_count 
    PWL= Pnouns
    #BWL = []
    #PWL = []
    

    F=[]     #freq of words  which appeared in nodes WRT document
    T=[]     #freq of words  which appeared in nodes WRT title
    K=[]     #freq of words  which appeared in nodes WRT keyword list 
    B=[]     #freq of words which appeared in nodes WRT Bi_Gram list 
    P=[]     #list of detection the proper noune 
    FW=[]    
    Xt,Xk,Xbi,Xp,Xb=1,1,1,1,1
    weighted_node=[]
    
    
    for i in node_list:
        flag=0
    
        for j in FWL:
            if(i==j[0]):
                f=j[1]
                F.append(f)
                flag=1
                
        if(flag!=1): 
            F.append(0) 
        
 

    for i in node_list:
        flag=0
        for j in TWL:
            if(i==j[0]):
                t=j[1]
                T.append(t)
                flag=1
        if(flag!=1): 
             T.append(0)
                
        
    for i in node_list:
        flag=0
        for j in KWL:
            if(i==j[0]):
                k=j[1]
                K.append(k)
                flag=1
        if(flag!=1): 
            K.append(0)
            
            
            
    for i in node_list:
        flag=0
        for j in BWL:
            if(i==j[0]):
                b=j[1]
                B.append(b)
                flag=1
        if(flag!=1): 
            B.append(0)
            
            
    for i in node_list:
        flag=0
        for j in PWL:
            if(i==j):
                P.append(1)
                flag=1
        if(flag!=1): 
            P.append(0)             
    
    
    
    for i in FWL:
        fw=i[1]
        FW.append(fw)
    
    if FW == []:
        avrage_word_frequency= 0
        median_word_frequency= 0 
    elif len(FW) != 0:
        avrage_word_frequency=st.mean(FW)
        median_word_frequency=st.median(FW) 
    else:
        avrage_word_frequency= 0
        median_word_frequency= 0 
        
    AW=abs(avrage_word_frequency-median_word_frequency)
    
    for i in range(len(node_list)):
        WN=F[i]+Xt*AW*T[i]+Xk*AW*K[i]+Xb*AW*B[i]+Xp*AW*P[i]
        weighted_node.append(WN)
        node_weight_dict[node_list[i]] = WN
    #print(K)
    #print (F,T,K,FW,avrage_word_frequency,median_word_frequency)    
    #print(weighted_node) 
    #print(node_weight_dict)
    return node_weight_dict,weighted_node

## functions for algorithm 2

In [17]:
#def get_nodes_list():
    #return All_nodes,weighted_node
import json
from json import dumps


#edge between source and distenation
def get_edge(source,distenation):
    return edgeget[source,distenation]


#distination node list
def getweights_of_dist_nodes(source_destinations,sourcenode,All_nodes_ww):  
    x=source_destinations[sourcenode]
    distenation_node_list={}
    weights=[]
    nothing=0
    for i in range(len(x)):
        if ((x[i])==('E#')) or ((x[i]) not in json.dumps(All_nodes_ww)) or ( x[i] not in All_nodes_ww):
          nothing=1
        else:
          distenation_node_list[x[i]]=All_nodes_ww[x[i]]
          weights.append((All_nodes_ww[x[i]]))
        if len(weights) != 0:
            nothing=0
    return distenation_node_list,weights,nothing

## algorithm 2 get candidate edges

In [18]:
#check for candidate edges between a source and a destination given a criteria 
def get_candidate_edges(source_destinations,node_listx,wxx,out_carteria,All_weight,input_nodes,All_SD):
    c=[]
    xxx = []
    no_list,totalweights=node_listx,All_weight

    source_node_weight=0
    distination_node_weight=0
    average_of_the_node_weight=np.average(totalweights)#(np.sum(node_list)/number_of_nodes)
    median_of_the_node_list_weight=np.median(totalweights)
  
    if average_of_the_node_weight>median_of_the_node_list_weight:#can be replaced with source=max(avg,median)
        source_node_weight=average_of_the_node_weight
    else:
        source_node_weight=median_of_the_node_list_weight

    for node,weight in no_list.items():
        
        if weight>=source_node_weight:#بيعرف اذا كانت بدايه الجمله ولا لا 
            outnodes1,outnodesweights,flagnothing=getweights_of_dist_nodes(source_destinations,node,no_list)#فنكشن جوا الجي بتجبلي كل الديستناشن بتاعت النودواختار بينهم هسقط مين
            outnodes2,nodes2weights,flag2nothing=getweights_of_dist_nodes(All_SD,node,input_nodes)
            outnodes1=[(i,j) for (i,j) in outnodes1.items() if j!=0 ] 
            outnodes2 =[(i,j) for (i,j) in outnodes2.items() if j!=0 ] 
        
            if flagnothing==0:
                #destination nodes exsist
                #get their average and max weights for avg and max criterias
                average_weight=np.average(nodes2weights)
                max_weight=max(nodes2weights)
            else:
                #no destination nodes 
                average_weight=0
                max_weight=0
            # get threshold of destination weights according to criteria 
            if (out_carteria =="avg"):
                distination_node_weight=average_weight
            elif (out_carteria =="node_avg"):
                distination_node_weight=source_node_weight
            elif (out_carteria=="max"):
                distination_node_weight=max_weight
            else:
                distination_node_weight=average_weight


            if (bool(outnodes1)==False):
                # no destination nodes 
                nothing=1  
            else: 
                # check for the candidate edges
                for out,weight_of_out_node in outnodes1:
                    if weight_of_out_node>=distination_node_weight and flagnothing==0:
                        c.append(get_edge(node,out))# append the edge
                        xxx.append([node,out])

    cx=[]
    for i in range(0,len(c)):
        cx.append(c[i][0])
    return cx
  

## functions needed for post-processing

In [20]:

def sentence_order(count_summ, index):
    return(count_summ - index)/count_summ

def compute_rank(candidate_summary):#, W, core_words_list, bigrams_list, lsa_weight):
    ranks = {}
    candidate_summary = reorder(candidate_summary)
    
    for i, sent in enumerate(candidate_summary):
        
        rank = sentence_order(len(candidate_summary), i)
        ranks[sent] = rank
        #print("index",i,"sentence",sent,"rank",rank)
    sort_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)    
    return ranks   

def sent_vectorizer(sent, model):
    sent_vec =[]
    numw = 0
    for w in sent:
        try:
            if numw == 0:
                sent_vec = model.wv[w]
            else:
                sent_vec = np.add(sent_vec, model.wv[w])
            numw+=1
        except:
            pass
     
    return np.asarray(sent_vec) / numw

def get_words_count(sent):
    len_sent = 0
    for i in sent.split(" "):
        len_sent = len_sent +1
    return len_sent

# reorder the summary based on the order from the input text
def reorder(summary):
    index_summ = []
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    order = dict()
    final_order = []
    for sent in summary:
        if sent in value_sent:
            index_sent = value_sent.index(sent)
            index_summ.append(index_sent)
            order[index_sent] = sent
            
    for i in sorted(order):
        final_order.append(order[i])
    #print(sorted(index_summ))
    return final_order

def concat(final_order):
    return("\n".join(final_order))



In [21]:
def sorted_cluster(x, model=None):
    if model == None:
        model = KMeans()
    model = sorted_cluster_centers_(model, x)
    model = sorted_labels_(model, x)
    return model

def sorted_cluster_centers_(model, x):
    model.fit(x)
    new_centroids = []
    magnitude = []
    for center in model.cluster_centers_:
        magnitude.append(np.sqrt(center.dot(center)))
    idx_argsort = np.argsort(magnitude)
    model.cluster_centers_ = model.cluster_centers_[idx_argsort]
    return model

def sorted_labels_(sorted_model, x):
    sorted_model.labels_ = sorted_model.predict(x)
    return sorted_model

## post processing and algorithm 4

In [23]:
def post_processing(list_of_candidates_summeries):
    candidate_summary = list_of_candidates_summeries
    #print('candidate summ    ',candidate_summary)
    sentences_ranks = compute_rank(candidate_summary)
   
    sentences_ordered =  [key for key,value  in sentences_ranks.items()]
    summary_length =0
    visited_sentences = []
    sentences = sentences_ordered

    tfidfvect = TfidfVectorizer(stop_words='english')
    X = tfidfvect.fit_transform(sentences)
      
    if len(sentences) <1:
        n_clusters = len(sentences)
    else:
        n_clusters = 1
    clf = KMeans(n_clusters = n_clusters,init = 'k-means++', max_iter = 1000)
    labels = clf.fit_predict(X)
    clustered_sentences =[[] for i in range(5)]

    cluster_map = pd.DataFrame()
    cluster_map['data_index'] = sentences
    cluster_map['cluster'] = clf.labels_
    #print(cluster_map)
    return n_clusters,cluster_map
n_clusters,cluster_map = post_processing(list_of_candidates_summeries)

def algorithm_4(summary_length,threshold,max_words_limit):
    candidate_summary = list_of_candidates_summeries
    summary_length = 0
    while (len(visited_sentences) < len(candidate_summary)) and ((summary_length)< max_words_limit) :
        for i in range(n_clusters):
            clustered_sentences = cluster_map[cluster_map.cluster == i]['data_index'].tolist()
            
            if summary_length >= (max_words_limit+threshold):
                break
            for sent in clustered_sentences:
                if sent not in visited_sentences :
                    temp_length = summary_length + get_words_count(sent)
                    if temp_length < max_words_limit :
                        summary_length = temp_length
                        final_summary.append(sent)
                        visited_sentences.append(sent)
                        break
                    elif(temp_length >= max_words_limit) and (temp_length <= (max_words_limit + threshold)):
                        summary_length = temp_length
                        final_summary.append(sent)
                        visited_sentences.append(sent)
                        break
                    elif temp_length > (max_words_limit + threshold):
                        visited_sentences.append(sent)
                        
    return final_summary



## function to calculate Rouge

In [25]:
def get_score(outsumm,Summary):
    score={}

    #generated_summary=" In a dramatic finish, the San Francisco 49ers football team won the Super Bowl on Sunday with a 20–16 victory over tCincinnati Bengals. Joe Montana's 10-yard touchdown pass to John Taylor with 34 seconds left provided the winningmargin. The victory was achieved through the brilliance of Montana and Jerry Rice, the wide receiver who caught 12passes for 222 yards, both Super Bowl records. It also gave 49ers Coach Bill Walsh his third Super Bowl win. The firsttouchdown didn't come until 44 minutes into the 60-minute game, when Cincinnati's Stanford Jennings took a kickoff 93yards for a score"
    generated_summary = outsumm
    #refrence1="The San Francisco 49ers won the Super Bowl Sunday with a dramatic 20–16 victory over the Cincinnati Bengals. JoeMontana's 10-yard touchdown pass to John Taylor with 34 seconds remaining provided the win. The pass gave Montana aSuper Bowl record with 357 yards passing. The victory was achieved through the brilliance of Montana and Jerry Rice, thewide receiver who caught 12 passes for 222 yards, both Super Bowl records. Rice was named the game's most valuableplayer. It was the fifth straight win for a National Football Conference team and the third Super Bowl win for Coach Bill"
    #refrence2="Joe Montana's 10-yard touchdown pass to John Taylor with 34 seconds left in the game gave the San Francisco 49ers a20–16 Super Bowl victory over the Cincinnati Bengals on Sunday. The touchdown gave Montana a Super Bowl record with357 yards passing. Wide receiver Jerry Rice caught 12 passes for 222 yards, both Super Bowl records, and was named thegame's most valuable player. It was the third Super Bowl win for 49ers Coach Bill Walsh. It was the fifth straight win for theNFC team and the most dramatic. The previous four had an average score of 41–1"

    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeLsum'], use_stemmer=True)
    s_R1 =[]
    s_R2 = []
    s_RL = []
    for summ in Summary:
        score = scorer.score('\n'.join(summ) ,generated_summary)
        s_R1.append(score['rouge1'][1])
        s_R2.append(score['rouge2'][1])
        s_RL.append(score['rougeLsum'][1])

    scores = [max(s_R1), max(s_R2) , max(s_RL)]

    return scores


## functions needed for graph

In [27]:
# Add a vertex to the dictionary
def add_vertex(v):
  global graph
  global vertices_no
  if v not in graph:
    vertices_no = vertices_no + 1
    #print("vertices_no ",vertices_no, v)
    graph[v] = []

# Add an edge between vertex v1 and v2 with edge weight e
#v1 is the source node and v2 is the distenation node
edgeget={}
def add_edge(v1, v2, e_order, e_value):
  global graph
  # Check if vertex v1 is a valid vertex
  if ((v1 in graph) and (v2 in graph)):
    #print(v1,v2,e_order,e_value)  
    temp = [v2, e_order, e_value]
    edgeget[v1,v2]=[e_value]
    graph[v1].append(temp)

# Print the graph
def print_graph():
  global graph
  for vertex in graph:
    for edges in graph[vertex]:
      #print("evet")
      print(vertex, " -> ", edges[0], "    edge order:", edges[1], "    edge value:", edges[2])

# get the destinations
def get_destinations(source):
    global graph
    distinations=[]
    if(source in graph):
        for elements in graph.get(source):
            if(elements[0] not in distinations):
                distinations.append(elements[0])
    return distinations

# get the destinations
def get_edges(source,dist):
    global graph
    Edge=""
    if(source in graph and dist in graph):
        for elements in graph.get(source):
            if(elements[0]==dist):
                Edge= Edge + elements[2]
    return Edge


## iterative part 

In [28]:
def iter_graph(can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria):
    All_SD = {}
    input_nodes = []
    sent_nodes = {}      
    nodes_list = []
    input_text = candidate_summ
    last_summ_len = candidate_summ_len
    Last_summ = candidate_summ
    sent_idx = can_edges
    can_edges=[]
    List_of_sent = list_pre_cansumm
    list_pre_cansumm = []
    list_of_candidates_summeries = []
    All_nodes=[]
    nodes_list = []
    # driver code
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    #print(S.values())
    index = 0
    for sent_index, sentence in zip(sent_idx,List_of_sent):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""
    #    print(sentence)
        Words=tokenizer.tokenize(sentence)
    #    print(Words)
        Types=pos_tag(Words)
    #    print(Types)
        for Type in Types:
            if (Type[1]=='NN' or Type[1]=='NNP' or Type[1]=='NNS' or Type[1]=='NNPS' or Type[1]=='VB' or Type[1]=='VBG' or Type[1]=='VBD' or Type[1]=='VBZ'):
            #            print(Type[0])
                nodes_list.append(Type[0])
                edges_list.append("")
            else:
                edges_list.append(Type[0])
                nodes_list.append("")
        #print("nodes_edges: ",nodes_list)
        #print("edges_nodes: ",edges_list) 


        list_of_nodes =[]
        LIST_OF_NODES=[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i
        #print("final node= ",final_node)

        list_of_nodes.remove("S#")    
        add_vertex("E#")
        #list_of_nodes.append("E#")
        nodes_list.append("E#")
        edges_list.append("")

        counter =0
        order =0
        
        for j in nodes_list :
            ##print("evet ", nodes_list[counter],counter,nodes_list[37])
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break
            #print("counter ",counter)
        #print("order= ",order) 
        #add_edge(final_node, "E#", order, "")
        
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)

        #for s in LIST_OF_NODES:
            #if(s not in source_destinations.keys()):
                #source_destinations[s]=get_destinations(s)
            #else:
                #source_destinations[s]= source_destinations[s]+ get_destinations(s)
        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)
            
        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)
        #print ("graph: ", graph)
        #
        
        #****************************************************************************
        input_nodes = input_nodes + list_of_nodes
        sent_nodes[index] = (source_destinations,list_of_nodes)
    
        #****************************************************************************
        graph = {}
        source_destinations={}
        index = index + 1
        
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes)
    All_weight = list_of_weights
    
    index = 0
    for sent_index, sentence in zip(sent_idx,List_of_sent): 
        (source_destinations,list_of_nodes) = sent_nodes[index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])


        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        
        if len(list_of_can_edges)>=1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[index] and (List_of_sent[index].strip()) ):
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[index])
                can_edges.append(sent_index)

        index = index + 1
        
    candidate_summ = reorder(list_of_candidates_summeries)
    candidate_summ_len = 0
    for index ,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + get_words_count(sent)

    return list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges

## function to get index of a sentence

In [29]:
def get_index(sent):
    key_sent = list(keyvalue.keys())
    value_sent = list(keyvalue.values())
    index_sent = value_sent.index(sent)
    return index_sent    

## final function
## iterates over all data in DUC2002 and gets Rouge results

In [30]:
import os
#path = "DucDataset/DUC2001_Summarization_Documents/data/testtraining/duc2002testtraining"
#files = os.listdir(path)
final_scores =[]
final_scores_1=[]
final_scores_2=[]
final_scores_L=[]
final_scores_1F=[]
final_scores_2F=[]
final_scores_LF=[]
final_scores_11 = []
final_scores_21 =[]
final_scores_L1 = []

final_scores_12 = []
final_scores_22 = []
final_scores_L2 = []



for i in Text_in:
    can_edges = []
    
    title,keywords,keyvalue, key_value_list,Summary, lang_Dict = read_input(Text_in[i], i) 
    out_carteria="avg"
    
    (title,keywords,Sentences_list,final_input,all_bigrams,bi_count_list,word_freq
         ,all_words,finalwordfreq,List_of_sent,title_word_freq,keyword_word_freq) = preprocess()
    #print(finalwordfreq)
    #print(keywords)
    #print(len(Sentences_list)-1)
    Last_summ = keyvalue.values()
    last_summ_len = 0
    for value in keyvalue.values():
        last_summ_len = last_summ_len + get_words_count(value)
    
    
    All_nodes=[]
    list_of_candidates_summeries=[]
    list_pre_cansumm = []
    ##asume
    #S={0:"Embedded software development, that is, the development of embedded software, such as used for controlling consumer products, requires the development process to be integrated with the development of the controlled physical product"}

    ##tokanization  
    tokenizer = TreebankWordTokenizer()

    ##lemmatization
    lemmatizer = WordNetLemmatizer() 

    # Add a vertex to the dictionary
    def add_vertex(v):
      global graph
      global vertices_no
      if v not in graph:
        vertices_no = vertices_no + 1
        #print("vertices_no ",vertices_no, v)
        graph[v] = []

    # Add an edge between vertex v1 and v2 with edge weight e
    #v1 is the source node and v2 is the distenation node
    edgeget={}
    def add_edge(v1, v2, e_order, e_value):
      global graph
      # Check if vertex v1 is a valid vertex
      if ((v1 in graph) and (v2 in graph)):
        #print(v1,v2,e_order,e_value)  
        temp = [v2, e_order, e_value]
        edgeget[v1,v2]=[e_value]
        graph[v1].append(temp)

    # Print the graph
    def print_graph():
      global graph
      for vertex in graph:
        for edges in graph[vertex]:
          #print("evet")
          print(vertex, " -> ", edges[0], "    edge order:", edges[1], "    edge value:", edges[2])

    # get the destinations
    def get_destinations(source):
        global graph
        distinations=[]
        if(source in graph):
            for elements in graph.get(source):
                if(elements[0] not in distinations):
                    distinations.append(elements[0])
        return distinations

    # get the destinations
    def get_edges(source,dist):
        global graph
        Edge=""
        if(source in graph and dist in graph):
            for elements in graph.get(source):
                if(elements[0]==dist):
                    Edge= Edge + elements[2]
        return Edge

    # driver code
    graph = {} 
    # stores the number of vertices in the graph
    vertices_no = 0    
    S = List_of_sent
    source_destinations={}
    #print(S.values())
    input_nodes = []
    sent_nodes = {}
    Pnouns = []
    for sent_index, sentence in enumerate(List_of_sent.values()):
        nodes_list = ["S#"]
        edges_list = [""]
        counter =0
        edge_value =""
        
        Words=tokenizer.tokenize(sentence)
        
        Types=pos_tag(Words)
    #    print(Types)
        for Type in Types:
            if (Type[1]=='NN' or Type[1]=='NNP' or Type[1]=='NNS' or Type[1]=='NNPS' or Type[1]=='VB' or Type[1]=='VBG' or Type[1]=='VBD' or Type[1]=='VBZ'):
            #            print(Type[0])
                nodes_list.append(Type[0])
                edges_list.append("")
                if(Type[1] == 'NNP'):
                    Pnouns.append("")
            
            else:
                edges_list.append(Type[0])
                nodes_list.append("")
        #print("nodes_edges: ",nodes_list)
        #print("edges_nodes: ",edges_list) 


        list_of_nodes =[]
        for i in nodes_list:
            if(i != ""):
                add_vertex(i)
                list_of_nodes.append(i)
                final_node =i
        #print("final node= ",final_node)

        list_of_nodes.remove("S#")    
        add_vertex("E#")
        #list_of_nodes.append("E#")
        nodes_list.append("E#")
        edges_list.append("")

        counter =0
        order =0
        
        for j in nodes_list :
            ##print("evet ", nodes_list[counter],counter,nodes_list[37])
            if(nodes_list[counter+1] !=""):
                add_edge(nodes_list[counter], nodes_list[counter+1], order, "")
                order =order +1
            else:
                x= counter
                y= edges_list[counter+1]
                while(nodes_list[counter+1] =="" and nodes_list[x] !=""):
                    counter =counter +1
                    if(counter ==len(nodes_list)-2):
                        y= y+" "+ edges_list[counter+1]
                add_edge(nodes_list[x], nodes_list[counter+1], order, y)
                order =order +1
            counter =counter +1
            if(counter == len(nodes_list)-1):
                break
            #print("counter ",counter)
        #print("order= ",order) 
        #add_edge(final_node, "E#", order, "")
        LIST_OF_NODES=[]
        for k in list_of_nodes:
            if(k not in LIST_OF_NODES):
                LIST_OF_NODES.append(k)

        #for s in LIST_OF_NODES:
            #if(s not in source_destinations.keys()):
                #source_destinations[s]=get_destinations(s)
            #else:
                #source_destinations[s]= source_destinations[s]+ get_destinations(s)
        
        for s in LIST_OF_NODES:
            if(s not in source_destinations.keys()):
                source_destinations[s]=get_destinations(s)
            else:
                source_destinations[s]= source_destinations[s]+ get_destinations(s)

        for s in LIST_OF_NODES:
            if(s not in All_SD.keys()):
                All_SD[s] = get_destinations(s)
            else:
                All_SD[s] = All_SD[s] + get_destinations(s)
        #print ("graph: ", graph)
        #
        #print_graph()
        #****************************************************************************
        input_nodes = input_nodes + list_of_nodes
        sent_nodes[sent_index] = (source_destinations,list_of_nodes)

        #****************************************************************************
        graph = {}
        source_destinations={}
        #print("list_of_nodes ",list_of_nodes)
        #print("sentence =",sentence)
        #All_nodes = All_nodes + list_of_nodes
        #****************************************************************************
    input_nodes = list(set(input_nodes))
    dic_of_nodes_and_weights,list_of_weights=get_weights(input_nodes)
    All_weight = list_of_weights
    for sent_index, sentence in enumerate(S.values()): 
        (source_destinations,list_of_nodes) = sent_nodes[sent_index]
        nodes_weights = {}
        weights = []
        for node in list_of_nodes:
            nodes_weights[node] = dic_of_nodes_and_weights[node]
            weights.append(dic_of_nodes_and_weights[node])

        list_of_can_edges=get_candidate_edges(source_destinations,nodes_weights,weights,out_carteria,All_weight,dic_of_nodes_and_weights,All_SD)
        
        if len(list_of_can_edges)>=1:#we will make it a variable depends on the length of the sentence
            if(List_of_sent[sent_index] and (List_of_sent[sent_index].strip()) ):
                list_of_candidates_summeries.append(keyvalue[sent_index])
                list_pre_cansumm.append(List_of_sent[sent_index])
                can_edges.append(sent_index)
                

    max_words_limit= 100
    threshold = 15
    final_summary = []
    visited_sentences = []
    summary_length = 0
        
    candidate_summ = reorder(list_of_candidates_summeries)
    
    candidate_summ_len = 0
    for index,sent in enumerate(candidate_summ):
        candidate_summ_len = candidate_summ_len + get_words_count(sent)
    

    while(True):
        if(candidate_summ_len > (max_words_limit + threshold)):
            if(candidate_summ_len != last_summ_len):
                list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph(can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria) 

            elif(out_carteria != "max"):
                out_carteria="max"
                list_pre_cansumm,candidate_summ,candidate_summ_len,Last_summ,last_summ_len,can_edges = iter_graph(can_edges,list_pre_cansumm,candidate_summ,candidate_summ_len,out_carteria)
            else:
                list_of_candidates_summeries = candidate_summ
                break
        elif(candidate_summ_len < (max_words_limit)):
            candidate_summ = Last_summ
            list_of_candidates_summeries = candidate_summ
            break
        else:
            list_of_candidates_summeries = candidate_summ
            break
            
    n_clusters,cluster_map = post_processing(list_of_candidates_summeries)
    final_summary = algorithm_4(summary_length,threshold,max_words_limit)
    final = reorder(final_summary)
    outsumm = concat(final)
    scores = get_score(outsumm,Summary)


    ##this avrage with respect to Recall

    final_scores_1.append(scores[0])
    final_scores_2.append(scores[1])
    final_scores_L.append(scores[2])
    #print(scores['rouge(1)'][0],"   ",scores['rouge(2)'][0], "  ",scores['rouge(L)'][0])

avrage_1=st.mean(final_scores_1)
avrage_2=st.mean(final_scores_2)
avrage_L=st.mean(final_scores_L)
print(avrage_1,avrage_2,avrage_L)


0.5220808625952076 0.27511553829391766 0.48183647921283745
